In [2]:
import sys
import cv2 as cv
import random
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.spatial import Delaunay
from PIL import Image, ImageOps, ImageDraw

In [3]:
#1) Load Image-------------------------------------
img_name = 'beatles_-_abbey_road.jpg'
img = Image.open('beatles_-_abbey_road.jpg')
width, height = img.size



In [121]:
#2) Determine probability that pixel gets point dropped-------------------------------------------------------

#a) Create two 2D arrays the same size as the img array
#b) Calculate the gradient of a pixel (based on some number of neighbors) [for now, we'll just say that |gradient| > x, but worth looking into whether Laplacian Zero crossing is greater than a threshold]
#c) Use this to find Laplacian Zero crossings of 3x3 patches
#d) Plot zero-crossings whose strength are greater than a threshold
    #Note: This means-- normalize strength of zero-crossing such that it's between [0, 1]-- this is percent chance it gets a point plotted
    #If there isn't a zero-crossing, the gradient will be low so either have a fixed probability or use some factor of the gradient
#e) Put these probabilities into 2D array

ddepth = cv.CV_16S
kernel_size = 3
src = cv.imread(cv.samples.findFile(img_name), cv.IMREAD_COLOR)
src = cv.GaussianBlur(src, (3, 3), 0)
src_grey = cv.cvtColor(src, cv.COLOR_BGR2GRAY)
dst = cv.Laplacian(src_grey, ddepth, ksize=kernel_size)
img_edges = cv.convertScaleAbs(dst)
#cv.imshow("", img_edges)
#cv.waitKey(0)
chances = img_edges/255
n = 50000

#Need tuple for some stuff and list for others

max_index = pd.DataFrame(data=chances).stack().sort_values(ascending=False).index[0:n]
x = max_index.get_level_values(1)
y = max_index.get_level_values(0)

coords = [(i, j) for i, j in zip(x, y)]
coord_list = [[i, j] for i, j in zip(x, y)]
triangulate_coords = [[j, i] for i, j in zip(x, y)]
#Coords indexes from the top, Delaunay indexes from the bottom

PIL indexes [col, row], or [y, x]
Delaunay indexes [x, y], starting from 0

Need to figure out how to unpack it to make it triangulate the correct way

In [122]:
#3) Create new PIL image and plot points

pic = Image.new(mode = "RGB", size=(width, height))
draw = ImageDraw.Draw(pic)
draw.point(coords)

In [123]:
#3)Delaunay Triangulation---------------------------------------------------------------------------------------
tri = Delaunay(coord_list)
#draw.polygon(tri.simplices)
new_coords = np.array(coords)
new_coords[tri.simplices][0].flatten().tolist()
for triangle in new_coords[tri.simplices]:
    draw.polygon(triangle.flatten().tolist())


pic.show()
#Note: We may use a different file here, and we're gonna use the neural network outlined in that research paper (find on Notion)

In [ ]:

#4)Color in the triangles ----------------------------------------------------------------------------------------

#Note: We could get the coordinates of the triangles and find the color of the center pixel, then draw a polygon